In [31]:
import os, json, pprint, re
import pandas as pd
import pickle as pkl
from collections import defaultdict
from matplotlib import pyplot as plt

## Paths

In [32]:
root_path = os.path.dirname(os.getcwd())
root_path

'C:\\Users\\david\\Projects\\epfl-pdm'

In [33]:
data_path = os.path.join(root_path, "data")
data_path

'C:\\Users\\david\\Projects\\epfl-pdm\\data'

In [34]:
raw_path = os.path.join(data_path, "raw")
raw_path

'C:\\Users\\david\\Projects\\epfl-pdm\\data\\raw'

In [35]:
clean_path = os.path.join(data_path, "clean")
clean_path

'C:\\Users\\david\\Projects\\epfl-pdm\\data\\clean'

In [36]:
group_paths = os.listdir(raw_path)
group_paths

['David', 'Gr1', 'Gr2', 'Gr3', 'Gr4', 'Gr5']

In [37]:
for p in group_paths:
    print(os.listdir(os.path.join(raw_path, p)))

['.ipynb_checkpoints', 'aufgabe1.log', 'aufgabe2.log', 'aufgabe3.log']
['aufgabe1.log', 'aufgabe2.log', 'aufgabe3.log']
['aufgabe1.log', 'aufgabe2.log', 'aufgabe3.log']
['aufgabe1.log', 'aufgabe2.log', 'aufgabe3.log']
['aufgabe1.log', 'aufgabe2.log', 'aufgabe3.log']
['Aufgabe I + II.log', 'Aufgabe I.log', 'Aufgabe III.log']


## Helpers

In [38]:
# Quickly replace multiple strings based on a dict
def replace_all(string, table):
    for k, v in table.items():
        string = string.replace(k, v)
    return string

# Non tail-recursive flatten operation
def flatten(original):
    entry = original.copy()
    children = entry.get("children")
    ret = [entry]
    
    if children:
        entry.pop("children")
        
        for c in children:
            ret += flatten(c)
            
    return ret

# Separate jsons from raw log file contents
def split_jsons(log):
    elems = log.split("}")
    mem, res = "", []

    for elem in elems:
        mem += elem + "}"

        if mem.count("{") == mem.count("}"):
            res.append(mem)
            mem = ""
            
    return res

## Parameters

In [39]:
replacements = {
    "beers-law-lab_en-phetio.html?phetioStandalone&phetioLog=console:983 ": "",
    "\u200b": "",
    "[Intervention] Unable to preventDefault inside passive event listener due to target being treated as passive. See <URL>": ""
}

## Processing files

In [40]:
data, temp = {}, {}

for p in group_paths:
    sub_data, sub_temp = {}, {}
    sub_path = os.path.join(raw_path, p)
    dirs = [d for d in os.listdir(sub_path) if d[0] != "."]
    
    for i, sub_p in enumerate(dirs):
        batch, batch_temp = [], []
        
        with open(os.path.join(sub_path, sub_p), "r", encoding="utf-8") as f:
            raw = split_jsons(replace_all(f.read(), replacements))
            print("Raw entries in", p, sub_p, ":", len(raw))
            
            for elem in raw:
                try:
                    batch += flatten(json.loads(elem))
                    batch_temp += [json.loads(elem)]
                except Exception as e:
                    print("\tDropped 1 invalid element")
                    print(elem)
                    
            print("\tFlattened entries :", len(batch))
        
        sub_data[i] = batch
        sub_temp[i] = batch_temp
        
    data[p] = sub_data
    temp[p] = sub_temp
    
with open("data.pkl", "wb") as f:
    pkl.dump(temp, f)

Raw entries in David aufgabe1.log : 1000
	Flattened entries : 3083
Raw entries in David aufgabe2.log : 1605
	Flattened entries : 4630
Raw entries in David aufgabe3.log : 524
	Flattened entries : 1628
Raw entries in Gr1 aufgabe1.log : 1000
	Flattened entries : 2714
Raw entries in Gr1 aufgabe2.log : 1000
	Flattened entries : 2595
Raw entries in Gr1 aufgabe3.log : 1000
	Flattened entries : 2557
Raw entries in Gr2 aufgabe1.log : 607
	Flattened entries : 1512
Raw entries in Gr2 aufgabe2.log : 1000
	Flattened entries : 3200
Raw entries in Gr2 aufgabe3.log : 1000
	Flattened entries : 3074
Raw entries in Gr3 aufgabe1.log : 743
	Flattened entries : 1869
Raw entries in Gr3 aufgabe2.log : 1000
	Flattened entries : 3025
Raw entries in Gr3 aufgabe3.log : 510
	Flattened entries : 1393
Raw entries in Gr4 aufgabe1.log : 1000
	Flattened entries : 3290
Raw entries in Gr4 aufgabe2.log : 3302
	Flattened entries : 10333
Raw entries in Gr4 aufgabe3.log : 4452
	Flattened entries : 13239
Raw entries in Gr5 Au

# JUNK TO BE SORTED OUT

In [ ]:
for k in ts.keys():
    print(k.split(".")[2:])

In [ ]:
chunks = [[s for s in flatten(f)] for f in temp["Gr4"][1]]
len(chunks)

#[".".join(s["phetioID"].split(".")[2:])

In [ ]:
def get_chunk_data(chunk):
    return [((".".join(e["phetioID"].split(".")[2:]), e["event"]), e["time"], e.get("parameters")) for e in chunk]

def get_chunk_head(chunk):
    return get_chunk_data(chunk)[0]

In [ ]:
#newmap = dict()

#for k, v in mapper.items():
#    p1, p2 = k
#    newmap.setdefault(p1, {})[p2] = v
    
#newmap

In [ ]:
#with open("entries.json", "w") as f:
#    json.dump(newmap, f)

In [ ]:
with open("entries.json", "r") as f:
    newmap = json.load(f)
    
newmap

In [ ]:
x = pd.DataFrame([(newmap[p1][p2], time, data) for (p1, p2), time, data in [get_chunk_head(c) for c in chunks]])
x.columns = ["name", "time", "data"]
x[["object", "action"]] = x["name"].apply(lambda a : pd.Series(a.split("-", 1)))
x = x[x["action"] != "drag"]
x = x.reset_index(drop=True)    
x["time"] -= x["time"][0]
x["time"] /= 1000

# Eliminate cut drag
drags = x[x["action"].str[:5] == "drag-"]
first_drag = drags.iloc[0]
last_drag = drags.iloc[-1]

if first_drag["action"][-3:] == "end":
    x = x.loc[int(first_drag.name) + 1:]

if last_drag["action"][-5:] == "start":
    x = x.loc[:int(last_drag.name)]

with open("test.pkl", "wb") as f:
    pkl.dump(x, f)
    
x

In [ ]:
list(set([a.split("-")[0] for a in x[0].unique()]))

In [ ]:
unique_chunks = []

for c in chunks:
    unseen = True
    data = get_chunk_head(c)
    
    for u in unique_chunks:
        if u == data:
            unseen = False
            break
    
    if unseen:
        unique_chunks.append(data)
        
len(unique_chunks)

In [ ]:
unique_chunks = []

for i in range(1, 6):
    for j in range(3):
        chunks = [[s for s in flatten(f)] for f in temp["Gr" + str(i)][j]]
        
        for c in chunks:
            unseen = True
            data = get_chunk_head(c)

            for u in unique_chunks:
                if u == data:
                    unseen = False
                    break

            if unseen:
                unique_chunks.append(data)

In [ ]:
# For 'TTandemDragHandler'
def getDragHandler(df, pid):
    # Filter on property
    df = df[df["phetioID"] == pid]
    
    # Filter on property
    df = df[df["phetioID"] == pid]
    
    # Exit if property absent
    if len(df) == 0:
        return None
    
    # Index on time
    df.set_index("time", inplace=True)
    
    # Remove unwanted columns
    df = df["parameters"].apply(pd.Series)
    
    return df[["x", "y"]]

# For 'TProperty' and 'TDerivedProperty'
def getProperty(df, pid):
    # Get start time
    init_time = df.loc[0]["time"]
    
    # Filter on property
    df = df[df["phetioID"] == pid]
    
    # Exit if property absent
    if len(df) == 0:
        return None
    
    # Index on time
    df.set_index("time", inplace=True)
    
    # Remove unwanted columns
    df = df["parameters"].apply(pd.Series)
    init_val = df.iloc[0]["oldValue"]
    df = df["newValue"]
    
    try:
        # Add starting value
        df.loc[init_time] = init_val
    except:
        pass
        # THIS IS TEMPORARY; THERE IS A BUG HERE
    
    return df.sort_index()

# For 'TToggleButton'
def getToggle(df, pid):
    return getProperty(df, pid).astype(int)

# For 'TPushButton', 'TRadioButton', 'TComboBox', 'TScreenButton', 'TTandemText' and 'TComboBox'
# FUTURE : EXTEND TO PLUS/MINUS BUTTONS
def getButton(df, pid):
    
    # Filter on property
    df = df[df["phetioID"] == pid]
    
    # Exit if property absent
    if len(df) == 0:
        return None
    
    # Index on time
    df.set_index("time", inplace=True)
    
    # Remove unwanted columns
    return df["event"]

def processComp(df, cType, pid):
    if cType == 'TTandemDragHandler':
        return getDragHandler(df, pid)
    elif cType == 'TToggleButton':
        return getToggle(df, pid)
    elif "Property" in cType:
        return getProperty(df, pid)
    else:
        return getButton(df, pid)

def processLog(df):
    uniques = df[["componentType", "phetioID"]].drop_duplicates()

In [ ]:
comps = {}

for e in data["Gr4"][2]:
    ct = e["componentType"]
    comps[ct] = comps.get(ct, 0) + 1
    
comps

In [ ]:
cts, evs, pids = set(), set(), set()
gen = ((d["componentType"], d["event"], d["phetioID"]) for k in data for k2 in data[k] for d in data[k][k2])
d = defaultdict(dict)

for c, b, a in gen:
    try:
        d[c][a] = list(set(d[c][a] + [b]))
    except:
        d[c][a] = [b]

categories = d
pprint.pprint(categories)

In [ ]:
gen = ((k, k2, k3) for k in categories for k2 in categories[k] for k3 in categories[k][k2])
list(gen)

In [ ]:
gen = ((k, k2, k3) for k in categories for k2 in categories[k] for k3 in categories[k][k2])
d = {}

for cType, _, event in gen:
    try:
        d[event] = list(set(d[event] + [cType]))
    except:
        d[event] = [cType]
        
pprint.pprint(d)

In [ ]:
gen = ((k, k2, k3) for k in categories for k2 in categories[k] for k3 in categories[k][k2])
d = {}

for cType, phetioID, _ in gen:
    try:
        d[cType] = list(set(d[cType] + [phetioID]))
    except:
        d[cType] = [phetioID]
        
pprint.pprint(d)

## TURN LASER ON / OFF

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.lightNode.button",
"componentType": "TToggleButton",
"event": "toggled"

LASER LIGHT (PASSIVE)

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.light.onProperty",
"componentType": "TProperty",
"event": "changed"

TRANSMITTANCE / ABSORBANCE INTERNAL VALUE (PASSIVE)

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.detector.valueProperty",
"componentType": "TDerivedProperty",
"event": "changed"

TRANSMITTANCE / ABSORBANCE DISPLAY (PASSIVE)

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.view.detectorNode.bodyNode.valueNode",
"componentType": "TTandemText",
"event": "textChanged"

## SWITCH TRANSMITTANCE / ABSORBANCE

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.detectorNode.bodyNode.transmittanceRadioButton",
OR
"phetioID": "beersLawLab.beersLawScreen.view.detectorNode.bodyNode.absorbanceRadioButton",
"componentType": "TRadioButton",
"event": "fired",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.detector.modeProperty",
"componentType": "TProperty",
"event": "changed",

## SWITCH PRESET / VARIABLE

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.presetWavelengthRadioButton",
OR
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.variableWavelengthRadioButton",
"componentType": "TRadioButton",
"event": "fired",

INTERNAL PROPERTY (PASSIVE)

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.variableWavelengthProperty",
"componentType": "TProperty",
"event": "changed",

## TWEAK WAVELENGTH

### SLIDER

#### DRAG START

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.wavelengthSlider.thumbInputListener",
"componentType": "TTandemDragHandler",
"event": "dragStarted",

#### DRAG

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.wavelengthSlider.thumbInputListener",
"componentType": "TTandemDragHandler",
"event": "dragged",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.light.wavelengthProperty",
"componentType": "TProperty",
"event": "changed",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.valueDisplay",
"componentType": "TTandemText",
"event": "textChanged",

#### DRAG END

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.wavelengthSlider.thumbInputListener",
"componentType": "TTandemDragHandler",
"event": "dragEnded"

### BUTTONS

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.wavelengthSlider.plusButton",
OR
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.wavelengthSlider.minusButton",
"componentType": "TPushButton",
"event": "fired",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.light.wavelengthProperty",
"componentType": "TProperty",
"event": "changed",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.valueDisplay",
"componentType": "TTandemText",
"event": "textChanged",

## MOVE RULER

### DRAG START

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.rulerNode.movableDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragStarted",

### DRAG

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.rulerNode.movableDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragged",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.ruler.locationProperty",
"componentType": "TProperty",
"event": "changed",

### DRAG END

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.rulerNode.movableDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragEnded",

## RESIZE CONTAINER

### DRAG START

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.cuvetteNode.cuvetteDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragStarted",

### DRAG

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.cuvetteNode.cuvetteDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragged",

### DRAG END

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.cuvetteNode.cuvetteDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragEnded",

## MOVE PROBE

### DRAG START

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.detectorNode.probeNode.movableDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragStarted",

### DRAG

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.detectorNode.probeNode.movableDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragged",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.detector.probe.locationProperty",
"componentType": "TProperty",
"event": "changed",

### DRAG END

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.detectorNode.probeNode.movableDragHandler",
"componentType": "TTandemDragHandler",
"event": "dragEnded",

### WHEN SNAP FOLLOWED BY

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.detector.probe.locationProperty",
"componentType": "TProperty",
"event": "changed",

With property y = 2

## SOLUTION SELECTION

### OPEN

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.comboBox",
"componentType": "TComboBox",
"event": "popupShown",

### CLOSE

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.comboBox",
"componentType": "TComboBox",
"event": "popupHidden",

### CHOOSE

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.comboBox",
"componentType": "TComboBox",
"event": "fired",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.solutionProperty",
"componentType": "TProperty",
"event": "changed",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.model.light.wavelengthProperty",
"componentType": "TProperty",
"event": "changed",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.view.wavelengthControls.valueDisplay",
"componentType": "TTandemText",
"event": "textChanged",

## TWEAK CONCENTRATION

### SLIDER

#### DRAG START

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.concentrationControl.slider.thumb.dragHandler",
"componentType": "TTandemDragHandler",
"event": "dragStarted",

#### DRAG

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.concentrationControl.slider.thumb.dragHandler",
"componentType": "TTandemDragHandler",
"event": "dragged",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.solutions.cobaltChloride.concentrationProperty",
"componentType": "TProperty",
"event": "changed",

#### DRAG END

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.concentrationControl.slider.thumb.dragHandler",
"componentType": "TTandemDragHandler",
"event": "dragEnded",

### BUTTONS

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.concentrationControl.slider.plusButton",
OR
"phetioID": "beersLawLab.beersLawScreen.view.solutionControls.concentrationControl.slider.minusButton",
"componentType": "TPushButton",
"event": "fired",

"eventType": "model",
"phetioID": "beersLawLab.beersLawScreen.solutions.cobaltChloride.concentrationProperty",
"componentType": "TProperty",
"event": "changed",

## RESET

"eventType": "user",
"phetioID": "beersLawLab.beersLawScreen.view.resetAllButton",
"componentType": "TPushButton",
"event": "fired",

In [ ]:
actions = {
    "concentration-drag-start":   ('view.solutionControls.concentrationControl.slider.thumb.dragHandler', 'dragStarted'),
    "concentration-drag":         ('view.solutionControls.concentrationControl.slider.thumb.dragHandler', 'dragged'),
    "concentration-drag-end":     ('view.solutionControls.concentrationControl.slider.thumb.dragHandler', 'dragEnded'),
    #"concentration-drag-start":   ('view.solutionControls.concentrationControl.slider.track.inputListener', 'dragStarted'),
    #"concentration-drag":         ('view.solutionControls.concentrationControl.slider.track.inputListener', 'dragged'),
    #"concentration-drag-end":     ('view.solutionControls.concentrationControl.slider.track.inputListener', 'dragEnded'),
    "concentration-plus":         ('view.solutionControls.concentrationControl.slider.plusButton', 'fired'),
    "concentration-minus":        ('view.solutionControls.concentrationControl.slider.minusButton', 'fired'),
    "container-drag-start":       ('view.cuvetteNode.cuvetteDragHandler', 'dragStarted'),
    "container-drag":             ('view.cuvetteNode.cuvetteDragHandler', 'dragged'),
    "container-drag-end":         ('view.cuvetteNode.cuvetteDragHandler', 'dragEnded'),
    "laser-toggle":               ('view.lightNode.button', 'toggled'),
    "probe-toggle-absorbance":    ('view.detectorNode.bodyNode.absorbanceRadioButton', 'fired'),
    "probe-toggle-transmittance": ('view.detectorNode.bodyNode.transmittanceRadioButton', 'fired'),
    "probe-drag-start":           ('view.detectorNode.probeNode.movableDragHandler', 'dragStarted'),
    "probe-drag":                 ('view.detectorNode.probeNode.movableDragHandler', 'dragged'),
    "probe-drag-end":             ('view.detectorNode.probeNode.movableDragHandler', 'dragEnded'),
    "ruler-drag-start":           ('view.rulerNode.movableDragHandler', 'dragStarted'),
    "ruler-drag":                 ('view.rulerNode.movableDragHandler', 'dragged'),
    "ruler-drag-end":             ('view.rulerNode.movableDragHandler', 'dragEnded'),
    "sim-start":                  ('', 'simStarted'),
    "sim-reset":                  ('view.resetAllButton', 'fired'),
    "solution-open":              ('view.solutionControls.comboBox', 'popupShown'),
    "solution-choose":            ('view.solutionControls.comboBox', 'fired'),
    "solution-close":             ('view.solutionControls.comboBox', 'popupHidden'),
    "wavelength-toggle-preset":   ('view.wavelengthControls.presetWavelengthRadioButton', 'fired'),
    "wavelength-toggle-variable": ('view.wavelengthControls.variableWavelengthRadioButton', 'fired'),
    "wavelength-drag-start":      ('view.wavelengthControls.wavelengthSlider.thumbInputListener', 'dragStarted'),
    "wavelength-drag":            ('view.wavelengthControls.wavelengthSlider.thumbInputListener', 'dragged'),
    "wavelength-drag-end":        ('view.wavelengthControls.wavelengthSlider.thumbInputListener', 'dragEnded'),
    "wavelength-drag-start":      ('view.wavelengthControls.wavelengthSlider.trackInputListener', 'dragStarted'),
    "wavelength-drag-end":        ('view.wavelengthControls.wavelengthSlider.trackInputListener', 'dragEnded'),
    "wavelength-plus":            ('view.wavelengthControls.wavelengthSlider.plusButton', 'fired'),
    "wavelength-minus":           ('view.wavelengthControls.wavelengthSlider.minusButton', 'fired'),
    "window-big":                 ('view.beersLawScreenSmallButton', 'fired'),
    "window-small":               ('view.beersLawScreenLargeButton', 'fired'),
}